In [1]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12

In [2]:
import numpy as np
from datasets import load_dataset
import json
import os
import wave
import collections
import pickle
import random

In [3]:
fnumbers = ['3148-2','3147-1','conf_2690_2690_00862467','conf_2688_2688_00862561']
wav_fnames = []
for f in fnumbers:
    for i in range(1,9):
        wav_fnames.append(f + f'_{i}.wav')

In [4]:
dataset_repo_test = "johnlohjy/imda_nsc_p3_test_noiseaugmented_reverb_1mcanteen_mini_5db"
dataset_test = load_dataset(dataset_repo_test, split='test', streaming=True, trust_remote_code=True)

(…)seaugmented_reverb_1mcanteen_mini_5db.py:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

In [5]:
RATE = 16000
chunk = 4096
no_voice_activity_threshold = 20
references = []
total_audio_len = 0
total_audio_len_extra_silences = 0


#####################CREATE CONTINOUS AUDIO ARRAY ONCE S.T ITS REPRODUCIBLE############
# Store all audio samples
all_audio_samples = []

for sample in dataset_test:
  # If its a sample we want
  if sample['path'].split('/')[1] in wav_fnames:
    # Get a single audio samples data and convert from float 64 to float 32
    # Append it to all_audio_samples
    audio_sample = sample["audio"]["array"].astype(np.float32)
    all_audio_samples.append(audio_sample)

    # Increment the audio length
    total_audio_len += len(audio_sample)/16000
    total_audio_len_extra_silences += len(audio_sample)/16000

    # Insert random lengths of silence (1s to 5s) at the end of the curr sample
    silence = np.zeros(random.randint(1, 5)*RATE, dtype=np.float32)
    all_audio_samples.append(silence)

    # Increment the total_audio_len_extra_silences with silence
    total_audio_len_extra_silences += len(silence)/16000

    # Add the reference
    references.append(sample["sentence"])

# Concatenate all audio samples (with silences)
audio_samples = np.concatenate(all_audio_samples)

# Add no_voice_activity_threshold chunks of silence to the concatenated audio samples so the lastest segment of audio will be saved
silence = np.zeros(chunk*(no_voice_activity_threshold+5), dtype=np.float32)
audio_samples = np.concatenate((audio_samples, silence))
total_audio_len_extra_silences += len(silence)/16000
#####################CREATE CONTINOUS AUDIO ARRAY ONCE S.T ITS REPRODUCIBLE############

In [6]:
######################SAVE CONTINOUS ARRAYS######################
with open(f"test_audio_{total_audio_len/60}_mins_reverb_1mcanteen_mini_5db.pkl", "wb") as f:
    pickle.dump(audio_samples, f)

'''
with open(f"test_references.pkl", "wb") as f:
    pickle.dump(references, f)

with open(f"test_wavfnames.pkl", "wb") as f:
    pickle.dump(wav_fnames, f)
'''
######################SAVE CONTINOUS ARRAYS######################

'\nwith open(f"test_references.pkl", "wb") as f:\n    pickle.dump(references, f)\n\nwith open(f"test_wavfnames.pkl", "wb") as f:\n    pickle.dump(wav_fnames, f)\n'

In [ ]:
print(f'The total audio length is {total_audio_len}')
print(f'The total audio length with silences is {total_audio_len_extra_silences}')

The total audio length is 825.1258124999998
The total audio length with silences is 957.5258124999998


In [ ]:
snr_wavfile_values = {}

with open("data_test_waves_snr.txt", "r") as file:
    for line in file:
        key, value = line.strip().split(' ')
        snr_wavfile_values[key] = value

snr_wavfile_counts = collections.defaultdict(int)

for fname in wav_fnames:
    snr_wavfile_counts[snr_wavfile_values[fname.split('.')[0]]] += 1

In [ ]:
snr_wavfile_counts

defaultdict(int, {'10': 10, 'none': 6, '5': 11, '15': 5})

In [ ]:
with open(f"test_snr_distribution.pkl", "wb") as f:
    pickle.dump(snr_wavfile_counts, f)

In [ ]:
total_audio_len/60

13.752096874999998

In [ ]:
916.5258124999998/60

15.27543020833333